## Pytorch 에서 nn.Conv2d

**Pytorch 에서 Connvolutional layer 기본 code**

==> torch.nn.Conv2d(in_channels, out_channels, kernel_size , stride=1, padding=0, bias = True)

ex) 입력 채널 1 , 출력 채널 1, 커널 크기 3x3 인경우

conv = nn.Conv2d(1,1,3)

**Output size 계산**

output size = (input size- filter size + (2*padding)) / stride +1

out.shape으로 torch Size 확인 가능

**Pooling**

- Max pooling

: 이미지 사이즈 줄이기 위해 사용

- Average pooling

: fully connected 연산 대체하기 위해 average pooling으로 대체 하는데에 사용

==> torch.nn.MaxPool2d(kernel_size, stride = None, padding)

## MNIST CNN Code

1. library 가져오기
2. GPU 사용 설정, random value를 위한 seed 설정
3. 학습에 사용되는 parameter 설정(learning_rate, training_epochs, batch_size, etc)
4. 데이터셋 가져오고(학습에 쓰기 편하게) loader 만들기
5. 학습 모델 만들기 (class CNN(torch.nn.Module))
6. Loss function (Criterion)을 선택하고 최적화 도구 선택 (optimizer)
7. 모델 학습 및 loss check(Criterion의 output)
8. 학습된 모델의 성능 확인

In [8]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
import torch.nn as nn

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [22]:
learning_rate = 0.001
training_epochs = 5
batch_size = 100

In [4]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [11]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [47]:
class CNN(nn.Module):

  def __init__(self):
    super(CNN, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1,32,kernel_size = 3, stride =1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(32,64, kernel_size =3, stride =1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.layer3 = nn.Sequential(
        nn.Conv2d(64,128, kernel_size =3, stride =1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.fc1 = nn.Linear(3*3*128, 625) # 7*7*64 를 maxpooling을 거치면 3*3*128이된다
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(625, 10 ,bias = True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    torch.nn.init.xavier_uniform_(self.fc2.weight) #초기화

  def forward(self, x):
    out = self.layer1(x) #layer1을 다 통과하고 나온 x
    out = self.layer2(out)
    out = self.layer3(out)

    out = out.view(out.size(0), -1) #batch size만큼 펼치는 과정
    #print(out.shape) # 1152 % 128 = 9 ---> 3*3 이 된다
    out = self.fc1(out)
    out = self.relu(out)
    out = self.fc2(out)
    return out

In [48]:
model = CNN().to(device) #모델 생성하기 
print(model)

#
value = torch.Tensor(1,1,28,28).to(device)
print((model(value)).shape)

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1152, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)
torch.Size([1, 10])


In [49]:
#loss function과 optimizer설정하기
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [50]:
#training
total_batch = len(data_loader) #전체 batch size

for epoch in range(training_epochs):
  avg_cost =0 #여기에 loss를 하나씩 담을것

  for X,Y in data_loader :  # X 는 (input)이미지, Y는 라벨을 받는다
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad() # 반듣시 넣기!! 초기화 하는 작업?
    hypothesis = model(X)

    cost = criterion(hypothesis, Y) #예측값과 실제값 사이 cost 계산
    cost.backward()
    optimizer.step()

    avg_cost += cost/total_batch

  print('[Epoch : {}] cost = {}'.format(epoch+1, avg_cost))

print('Learning Finished!')

[Epoch : 1] cost = 0.16483624279499054
[Epoch : 2] cost = 0.039947234094142914
[Epoch : 3] cost = 0.027737129479646683
[Epoch : 4] cost = 0.019490372389554977
[Epoch : 5] cost = 0.01831633597612381
Learning Finished!


In [51]:
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test), 1,28,28).float().to(device) #한번에 집어 넣으려고 펼친것 len(mnist_test)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test) #model에 X_test 한번에 넣음 / batch없이
  correct_prediction = torch.argmax(prediction,1) == Y_test
  accuracy = correct_prediction.float().mean()

  print('Accuracy:', accuracy.item())

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9891999959945679
